In [ ]:
using CommonDataFormat
using Dates
using Statistics
using CairoMakie
include("../../src/scripts/MAVEN_plot.jl")
using .MAVEN_plot

In [8]:
project_root = dirname(@__DIR__)
pwd()

"e:\\WorkSpace\\Tianwen\\project\\IonBeam"

In [3]:
omni_file = joinpath(pwd(), "..", "..", "Data", "MAVEN", "mvn_swi_l2_coarsearc3d_20210824_v02_r00.cdf")
#omni_file = joinpath(pwd(), "../Data/MAVEN/mvn_swi_l2_finearc3d_20210824_v02_r00.cdf")
data = CDFDataset(omni_file)
# data = load_cdf(omni_file)

CDFDataset{CommonDataFormat.NoCompression, Int64}
path: e:\WorkSpace\Tianwen\project\IonBeam\..\..\Data\MAVEN\mvn_swi_l2_coarsearc3d_20210824_v02_r00.cdf
variables:
  epoch : (10800,) support_data CDF_TIME_TT2000 [2021-08-24T00:00:05.130 … 2021-08-24T23:59:56.446]
  time_met : (10800,) support_data CDF_DOUBLE [6.830350196092682e8 … 6.831214108486176e8]
  time_unix : (10800,) support_data CDF_DOUBLE [1.6297632051307316e9 … 1.6298495964463248e9]
  atten_state : (10800,) support_data CDF_UINT1 [1 … 1]
  grouping : (10800,) support_data CDF_UINT1 [0 … 0]
  num_accum : (10800,) support_data CDF_INT2 [1 … 1]
  counts : (16, 4, 48, 10800) support_data CDF_FLOAT [0.0 … 0.0]
  diff_en_fluxes : (16, 4, 48, 10800) data CDF_FLOAT [0.0 … 0.0]
  geom_factor : (1,) support_data CDF_FLOAT [0.0039199996 … 0.0039199996]
  de_over_e_coarse : (1,) support_data CDF_FLOAT [0.15 … 0.15]
  accum_time_coarse : (1,) support_data CDF_FLOAT [0.0204 … 0.0204]
  energy_coarse : (48, 1) support_data CDF_FLOAT [23244

In [4]:
omni_file

"e:\\WorkSpace\\Tianwen\\project\\IonBeam\\..\\..\\Data\\MAVEN\\mvn_swi_l2_coarsearc3d_20210824_v02_r00.cdf"

In [6]:
include("../../src/scripts/MAVEN_plot.jl")
using .MAVEN_plot
a = ion_energy2v(1000, 1) # 测试函数

437.98770168075504

In [ ]:
# 时间范围设置
target_time_str = "2021-08-24T07:30:00"
start_time_str = "2021-08-24T06:00:00"
end_time_str = "2021-08-24T09:00:00"

# 将目标时间字符串解析为 DateTime 对象
target_dt = DateTime(target_time_str, dateformat"yyyy-mm-ddTHH:MM:SS")
start_dt = DateTime(start_time_str, dateformat"yyyy-mm-ddTHH:MM:SS")
end_dt = DateTime(end_time_str, dateformat"yyyy-mm-ddTHH:MM:SS")

# 数据处理
flux_4d = data["diff_en_fluxes"] # 维度: [Phi, Theta, Energy, Time]
energies = data["energy_coarse"][:]./1000 # 维度: [Energy]
epochs = data["epoch"][:]           # 维度: [Time] (需确保转换为 DateTime)
times_dates = DateTime.(epochs)  # 将 CDF 时间转换为 DateTime 对象数组

# 找到最接近的时间索引
target_val, target_idx = findmin(d -> abs(d - target_dt), times_dates)
start_val, start_idx = findmin(d -> abs(d - start_dt), times_dates)
end_val, end_idx = findmin(d -> abs(d - end_dt), times_dates)

# 降维处理
# 对角度维度求平均，得到 [Energy, Time]
flux_2d = dropdims(mean(flux_4d, dims=(1,2)), dims=(1,2))
# 转置为 [Time, Energy] 以适配 Makie
flux_final = flux_2d' 

# 激活CairoMakie
CairoMakie.activate!()

# 提取时间范围和通量数据
time_range = times_dates[start_idx:end_idx]
time_numeric = [datetime2unix(dt) for dt in time_range]
flux_at_target = flux_final[start_idx:end_idx, :]./1000

# 创建图形
fig = Figure(resolution=(1000, 600))
ax = Axis(fig[1, 1], 
          xlabel="Time (UTC)",
          ylabel="Energy (keV)")

# 创建热图
hm = heatmap!(ax, time_numeric, energies, flux_at_target, colormap=:viridis)

# 添加颜色条
Colorbar(fig[1, 2], hm; 
        label="Differential Energy Flux\neV/(cm2 s sr eV)", 
        width=20,
        ticks=LinearTicks(6),
        ticklabelalign=(:left, :center))

# 创建30分钟间隔的刻度
tick_times = start_dt:Dates.Minute(30):end_dt

# 将刻度时间转换为数值
tick_positions = [datetime2unix(t) for t in tick_times]

# 格式化刻度标签 (使用HH:MM格式)
tick_labels = Dates.format.(tick_times, "HH:MM")

# 设置刻度
ax.xticks = (tick_positions, tick_labels)

# 可选：旋转刻度标签以避免重叠
# ax.xticklabelrotation = π/6  # 30度旋转
# ax.xticklabelalign = (:right, :center)

# 设置y轴为对数刻度（如果能量范围很大）
ax.yscale = log10
ax.ylabel = "Energy (keV) [log scale]"

# 可选：添加网格线
ax.xgridvisible = true
ax.ygridvisible = true
ax.xgridstyle = :dash
ax.ygridstyle = :dash
ax.xgridcolor = (:gray, 0.2)
ax.ygridcolor = (:gray, 0.2)

# 保存图形
save("MAVEN_i_flux_heatmap_30min.png", fig, resolution=(1200, 800))

┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\USTC\.julia\packages\Makie\Vn16E\src\scenes.jl:264
┌ Warning: The keyword argument `resolution` for `save()` has been deprecated. Use `size` instead, which better reflects that this is a unitless size and not a pixel resolution.
└ @ Makie C:\Users\USTC\.julia\packages\Makie\Vn16E\src\display.jl:330


In [127]:
xd = start_dt .+ Dates.Minute.(30) .* range(0, 6)

7-element Vector{DateTime}:
 2021-08-24T06:00:00
 2021-08-24T06:30:00
 2021-08-24T07:00:00
 2021-08-24T07:30:00
 2021-08-24T08:00:00
 2021-08-24T08:30:00
 2021-08-24T09:00:00

In [124]:
using CairoMakie
CairoMakie.activate!()
time_range = times_dates[start_idx: end_idx]
time_numeric = [datetime2unix(dt) for dt in time_range]
flux_at_target = flux_final[start_idx: end_idx, :]./1000
fig = Figure()
ax = Axis(fig[1, 1])
hm = heatmap!(ax, time_numeric, energies, flux_at_target)
Colorbar(fig[1, 2], hm; label = "Differential Energy Flux (cm² s sr keV)⁻¹", width = 15)
xticks_pos = range(extrema(time_range)..., length=6)
# ax6.xticks = (xtk, Dates.format.(xd, "HH:MM"))
ax.xticks = (time_numeric, Dates.format.(time_range, "HH:MM"))
save("MAVEN_i_flux_heatmap.png", fig)

InexactError: InexactError: Int64(5.1012374397564805e13)

In [100]:
flux_at_target = flux_final[start_idx: end_idx, :]./1000

1351×48 Matrix{Float32}:
  0.0       5.39006   6.26637  14.3933   …  358.789   543.439   534.142
 15.6763   23.3823   17.0897   14.3932      597.883   776.346   647.743
 23.3655   12.1251    8.97291  12.4641      623.821   622.583   638.703
 18.313    16.0644    3.13319   0.0         824.034   942.504   762.639
 17.5322   11.2486    8.98166   3.1345      514.455   646.496   664.766
 17.4973   10.2041    6.27653   0.0      …  462.402   535.102   554.375
 14.8256   20.6536   14.8315   15.252       877.8     808.811   572.251
 25.61     13.9319    8.98307   3.1345      714.048   725.976   644.83
  8.55256   5.82662  10.2159   18.7518      554.436   496.739   693.477
 17.9429   21.1152    3.14335  11.2346      578.188   598.352   765.789
 16.0224    5.82662  20.3912    5.84085  …  581.426   732.099   662.866
  5.83491   9.39482   3.92919  15.699       745.903   543.393   543.468
 16.4845   14.795     5.83831  15.252       569.851   577.677   532.475
  ⋮                                     